## Facility Location Optimization for Dual-Capacity Drone Fleet in Last-Mile Delivery Operations


### Problem definition using Pyomo and solving it with Gurobi

In [1]:
import pyomo.environ as pyomo

# Initialization of the model
model = pyomo.ConcreteModel()

# Define Sets
F = range(3)    # Facilities Set
C = range(4)    # Customers Set
Ds = range(2)   # S-type Drones Set
Dl = range(1)   # L-type Drones Set

# Parameters
# Distance between facility i and customer j 
d = [[30, 50, 40, 60],  
     [45, 60, 35, 70],
     [50, 40, 60, 80]]

# Fixed cost of operating facility i
f = [50, 60, 70]

# Operational cost for serving customer j from facility i with an S-type  drone
cs = [[10, 15, 20, 25],  
      [12, 14, 18, 22],
      [8, 10, 14, 20]]

# Operational cost for serving customer j from facility i with an L-type drone
cl = [[20, 25, 30, 35], 
      [25, 30, 35, 40],
      [18, 22, 28, 34]]

# The capacity of each facility, the maximum number of drones it can support
K = [10, 12, 15]

# Weight of customer’s j package
W = [4, 6, 8, 9]

# Weight factor for S-type drones for the facilities
ws = 1

# Weight factor for L-type drones for the facilities
wl = 2

# Payload capacity of S-type drones
Ps = 5

# Payload capacity of L-type drones
Pl = 10

# Max Range of S-type drones
Rs = 50

# Max Range of L-type drones
Rl = 100

# Decision Variables 
# Binary variable. Facility i is opened (1) or closed (0)
model.x = pyomo.Var(F, domain=pyomo.Binary)
# Binary variable. If customer j is served by facility i using an S-type drone (1), otherwise (0)
model.ys = pyomo.Var(Ds, F, C, domain=pyomo.Binary)
# Binary variable. If customer j is served by facility i using an L-type drone (1), otherwise (0)
model.yl = pyomo.Var(Dl, F, C, domain=pyomo.Binary)

# Define Objective Function 
def objective_function_rule(model):
    fixed_costs = sum(f[i] * model.x[i] for i in F)
    variable_costs = sum(cs[i][j] * model.ys[s, i, j] + cl[i][j] * model.yl[l, i, j] for s in Ds for l in Dl for i in F for j in C)
    return fixed_costs + variable_costs

model.objective = pyomo.Objective(rule=objective_function_rule, sense=pyomo.minimize)

# Constraints
# 1. Each customer must be served by one facility and one type of drone.
def is_customer_served(model, j):
    return sum(model.ys[s, i, j] for s in Ds for i in F) + sum(model.yl[l, i, j] for l in Dl for i in F) == 1

model.customer_served = pyomo.Constraint(C, rule=is_customer_served)

# 2. Each facility has a total capacity limit for the number of drones it can support.
def is_facility_maxed(model, i):
    return sum(ws * model.ys[s, i, j] for s in Ds for j in C) + sum(wl * model.yl[l, i, j] for l in Dl for j in C) <= K[i] * model.x[i]

model.facility_capacity = pyomo.Constraint(F, rule=is_facility_maxed)

# 3. A customer can only be assigned to a facility that is open. 
def is_small_facility_open(model, s, i, j):
    return model.ys[s, i, j] <= model.x[i]

model.small_facility_open = pyomo.Constraint(Ds, F, C, rule=is_small_facility_open)

def is_large_facility_open(model, l, i, j):
    return model.yl[l, i, j] <= model.x[i]

model.large_facility_open = pyomo.Constraint(Dl, F, C, rule=is_large_facility_open)

# 4. The assigned drone type must be able to carry the package weight.
def payload_capacity_small_rule(model, s, i, j):
    return W[j] * model.ys[s, i, j] <= Ps

model.payload_capacity_small = pyomo.Constraint(Ds, F, C, rule=payload_capacity_small_rule)

def payload_capacity_large_rule(model, l, i, j):
    return W[j] * model.yl[l, i, j] <= Pl

model.payload_capacity_large = pyomo.Constraint(Dl, F, C, rule=payload_capacity_large_rule)

# 5. S-type drones can only serve customers within Rs while L-type drones can serve customers within Rl
def range_limit_small_rule(model, s, i, j):
    return d[i][j] * model.ys[s, i, j] <= Rs

model.range_limit_small = pyomo.Constraint(Ds, F, C, rule=range_limit_small_rule)

def range_limit_large_rule(model, l, i, j):
    return d[i][j] * model.yl[l, i, j] <= Rl

model.range_limit_large = pyomo.Constraint(Dl, F, C, rule=range_limit_large_rule)


# # 6. Each small drone should be assigned to exactly one customer
# def distinct_drone_assignment_small_rule(model, s):
#     return sum(model.ys[s, i, j] for i in F for j in C) == 1

# model.distinct_drone_assignment_small = pyomo.Constraint(Ds, rule=distinct_drone_assignment_small_rule)

# # Each large drone should be assigned to exactly one customer
# def distinct_drone_assignment_large_rule(model, l):
#     return sum(model.yl[l, i, j] for i in F for j in C) == 1

# model.distinct_drone_assignment_large = pyomo.Constraint(Dl, rule=distinct_drone_assignment_large_rule)

# # 7. Each drone should be assigned to exactly one facility
# def distinct_facility_assignment_small_rule(model, s):
#     return sum(model.ys[s, i, j] for i in F for j in C) == 1

# model.distinct_facility_assignment_small = pyomo.Constraint(Ds, rule=distinct_facility_assignment_small_rule)

# def distinct_facility_assignment_large_rule(model, l):
#     return sum(model.yl[l, i, j] for i in F for j in C) == 1

# model.distinct_facility_assignment_large = pyomo.Constraint(Dl, rule=distinct_facility_assignment_large_rule)


# Solve the model
solver = pyomo.SolverFactory('gurobi')
result = solver.solve(model, tee=True)

# Print Results
print("-----Facility Openings-----")
for i in F:
    print(f"Facility {i}: {'Open' if model.x[i]() > 0.5 else 'Closed'}")

print("-----Customer Assignments-----")
for j in C:
    assigned = False
    for i in F:
        for s in Ds:
            if model.ys[s, i, j]() > 0.5:
                print(f"Customer {j} is served by Facility {i} using Small Drone {s}.")
                assigned = True
        for l in Dl:
            if model.yl[l, i, j]() > 0.5:
                print(f"Customer {j} is served by Facility {i} using Large Drone {l}.")
                assigned = True
    if not assigned:
        print(f"Customer {j} is not assigned to any facility.")

print(f"Total Cost: {model.objective()}")


Set parameter Username
Academic license - for non-commercial use only - expires 2025-10-22
Read LP format model from file /tmp/tmp5xpigu5s.pyomo.lp
Reading time = 0.01 seconds
x1: 115 rows, 39 columns, 219 nonzeros
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 24.04.1 LTS")

CPU model: AMD Ryzen 5 3500U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 115 rows, 39 columns and 219 nonzeros
Model fingerprint: 0xcddb1fb1
Variable types: 0 continuous, 39 integer (39 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [8e+00, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 378.0000000
Presolve removed 99 rows and 24 columns
Presolve time: 0.01s
Presolved: 16 rows, 15 columns, 36 nonzeros
Variable types: 0 continuous, 15 integer (15 binary)
Found heuristic solution: 